In [ ]:
#Clear the dataset
rm(list=ls())

#Working directory
library(rstudioapi)
current_path <- getActiveDocumentContext()$path 
setwd(dirname(current_path ))
print( getwd())

#Upload the dataset
library(readxl)
US_Earnings <- read_excel("US_Earnings.xlsx")
View(US_Earnings) ##288 records, 2 features

#Data Exploration
plot(US_Earnings$Date,US_Earnings$Earnings) #In 2009 there was a drop in earnings
mean(US_Earnings$Earnings) #earning average = 32.164
summary(US_Earnings) #the minimum was 2.32, the maximum was 175.43


In [ ]:
#UNIVARIATE TIME SERIES

US_Earnings<-US_Earnings$Earnings
US_Earnings.TS<-ts(US_Earnings, start =c(1950,1), end =c(2021,3), frequency =4)


plot.ts(US_Earnings.TS, xlab='Time',ylab='Earnings', main='Earning Time Series') 


##1-AUGMENTED DICKEY FULLER (ADF) TEST FOR STATIONARITY OF TIME SERIES
library(tseries)
adf.test(US_Earnings.TS) #H0: non-stationary time series
                         #H1: stationary time series

#p-value=0.95 --> the series is not stationary!!

##DECOMPOSE
plot(decompose(US_Earnings.TS, type = 'additive'))
plot(decompose(US_Earnings.TS, type = 'multiplicative'))

#2-DIFFERENZIATION
ts_diff <- diff(US_Earnings.TS, differences = 1) # d=1;
ts.plot(ts_diff, col='blue')


adf.test(ts_diff) #p-value= 0.01 --> the series is stationary!!

library(forecast)
findfrequency(US_Earnings.TS) #5
findfrequency(ts_diff) #14

#3-SEASONALITY.
US.filt<-ma(US_Earnings.TS,order=4)
US.filt
lines(US.filt,col="red")
ggseasonplot(US.filt)

ggsubseriesplot(US_Earnings.TS)

In [ ]:
## ARMA MODELS 
dev.new()
par(mfrow=c(1,2))
Acf(ts_diff, main='ACF for Differenced Series') #autocorrelation test
Pacf(ts_diff, main='PACF for Differenced Series') #partial of autocorrelation test

#residue test: residues come from a normal
qqnorm(ts_diff)
qqline(ts_diff, col=2)
shapiro.test(ts_diff)

#Model estimation
ts_arma <- auto.arima(US_Earnings.TS, d=1, stepwise = F,approximation = F, trace = T, method = 'ML')
print(summary(ts_arma))
s=checkresiduals(ts_arma) #Ljung-Box test: p-value: 0.0814. do not show serial correlation,so they are independent!white noise


ts_arma1<- arima(US_Earnings.TS, order = c(1,1,4), method = "ML")
summary(ts_arma1)
LB=checkresiduals(ts_arma1)#Ljun-Box test: p-value:0.1823
BIC(ts_arma1)#1538


In [ ]:
## PREDICTION
tsforecasts <- forecast(ts_arma1, h = 8)
autoplot(tsforecasts)
summary(tsforecasts)

plot(tsforecasts$residuals)
accuracy(tsforecasts)


## INFERENCE ABOUT THE MODEL: TEST ML, WALD TEST & LM TEST

library(lmtest)
coeftest(ts_arma1)
qqnorm(ts_arma1$residuals) + qqline(ts_arma1$residuals)
